# def, fn

둘 중에 무엇을 선택하는 것은 사용자의 선택에 달림

def 는 python의 함수 선언 방식과 완전히 동일

fn은

In [ ]:
def addnumbers(a, b):
  c = a + b
  return c

print(addnumbers(2, 3))

In [ ]:
# 함수의 입력, 출력의 형태를 지정하여 함수를 정의할 수도 있음
def greet(name: String) -> String:
  let greeting = "Hello, " + name + "!"
  return greeting

print(greet("John"))

In [ ]:
# fn은 입력, 출력의 데이터 형태를 정의하여야 함
fn addnumbersfn(a: Int8, b: Int8) -> Int8:
  let c: Int8 = a + b
  return c

print(addnumbersfn(5, 3))

# Struct

AI 모델에서 struct는 연관된 데이터를 묶어서 함수로 하나의 unit으로 보낼 때 매우 유용하게 쓰임


In [ ]:
struct Animal:
  var age: Int
  var species: StringLiteral
  var sound: StringLiteral

  fn __init__(inout self, age: Int, species: StringLiteral, sound: StringLiteral):
    self.age = age
    self.species = species
    self.sound = sound

  fn printProps(self):
    print(self.age)
    print(self.species)
    print(self.sound)

var dog = Animal(2, "Dog", "woof")
var cat = Animal(1, "Cat", "meow")

dog.printProps()

dog.age = 3
dog.printProps()

# Memory 관리 방식

## inout, borrowed, owned



In [ ]:
# inout 이라는 키워드가 앞에 등장하면, var로 들어오면 그 변수를 변경할 수 있고, 변경할 수 없는 변수의 경우는 변경을 시도할 경우 시도할 수 없게 됨

fn add(inout a: Int, inout b: Int) -> Int:
  a = 6
  b = 7
  return a + b


fn main():
  var a: Int = 4
  var b: Int = 5
  print(add(a, b))

  # 아래와 같이 변경할 수 없는 변수를 놓고 함수에 적용을 시도하면 에러가 남...
  let a: Int = 4
  let b: Int = 5
  print(add(a, b))

main()

In [ ]:
# borrowed 이라는 키워드가 앞에 등장하면, 함수 안에서 가져온 변수의 값을 변경할 수 없음

fn add(borrowed a: Int, borrowed b: Int) -> Int:
  # a = 6 < 그래서 이와같이 외부에서 들어온 변수의 값을 바꿀 수 없게 됨
  # b = 7
  return a + b


fn main():
  # borrowed에서는 inout과 반대로 변경될 수 있는 값이 들어가면 경고를 띄움
  var a: Int = 4
  var b: Int = 5
  print(add(a, b))

  # 아래와 같이 변경할 수 없는 변수를 놓고 함수에 적용을 시도해야 함
  let a: Int = 4
  let b: Int = 5
  print(add(a, b))

main()

In [ ]:
# owned 이라는 키워드가 앞에 등장하면, 함수 안에서 가져온 변수를 임의로 변경할 수 있음!, 그리고 이는 변경할 수 없는 let으로 들어와도 마찬가지가 됨

fn add(owned a: Int, owned b: Int) -> Int:
  a = 6
  b = 7
  return a + b


fn main():
  # owned에서는 var로 들어오거나
  var a: Int = 4
  var b: Int = 5
  print(add(a, b))

  # let으로 들어와도 모두 변경할 수 있음!!!
  let a: Int = 4
  let b: Int = 5
  print(add(a, b))

main()

# Decorator

## @register_passable

int, float등의 기본 데이터들은 값이 복사되어 들어가고, reference들로 들어가지 않는 편인데
init 으로 초기화를 한 상태로, init 메소드 자체를 복사하고, 필요하지 않을 경우 메소드를 지울 필요가 있음

이렇게 임시로 쓰고, 지울 메소드 들은 다른 메소드들과 조금 다르게 초기 설정이 필요

또한 @register_passble을 쓰게 되면 함수를 선언할 때 inout등의 키워드를 쓸 수 없음


In [ ]:
@register_passable
struct Pair:
  var a: Int
  var b: Int

  fn __init__(one: Int, two: Int) -> Self:
    return self{a:one, b:two}

  fn __copyinit__(existing) -> Self:
    return Self{a:existing.a, b:existing.b}

fn test_pair():
  let x = Pair(5, 10)
  var y = x

  print(y.a, y.b)

  y.a = 10
  y.b = 20

  print(y.a, y.b)

test_pair()

In [ ]:
@register_passable("trivial") # trivial로 선언하면, 자동으로 __init__, __copyinit__을 선언해줌
struct Pair:
  var a: Int
  var b: Int


## @unroll


In [ ]:
for i in range(10):
  print(i)


In [ ]:
@unroll # 이렇게 처리하면...? unroll로 평행하게 돌리는 오브젝트를 여럿 돌린다고 함 > 병렬 처리 하는 방식
for i in range(10):
  print(i)

In [ ]:
@unroll(2) # 이렇게 하면 두 단계씩 끊어서 최종 5번의 반복이 진행

for i in range(10):
  print(i)

In [ ]:
# 위의 코드는 아래와 동일하게 동작함
for i in range(0, 10, 2):
  print(i)
  print(i+1)

## @always_inline

함수의 실행을 바로 할 수 있게 하나,
자주 불리는 함수에 사용하면 여러 복사본이 생성되고, 오버헤드가 커지는 단점이 있음

In [ ]:
@always_inline
fn add(a: Int, b: Int) -> Int:
  return a + b

print(add(1,2))

@always_inline("nodebug") # 이렇게 처리하면 always_inline과 똑같이 동작하나, 디버그용으로 띄워주는 내용들을 모두 제거하는 형태로 동작하기 때문에 빌드 시의 용량 크기를 일부 줄여주게 됨



## @parameter

In [ ]:
@parameter
if True:
  print("this will be included with the binary")
else:
  print("this will be eliminated at compile time")

In [ ]:
# dynamic하게 변경되는, 값을 인자로 받는 함수 등에 parameter를 적용하여 불러올 수 있도록 적용

fn use_closure[func: fn(Int) capturing -> Int](num: Int) -> Int:
  return func(num)


fn create_closure():
  var x = 1

  @parameter
  fn add(i: Int) -> Int:
    return x + i

  let y = use_closure[add](2)
  print(y)

create_closure()

## @capturing

In [ ]:
fn outer(func: fn() -> None):
  func()

fn call_it():
  @noncapturing # 이 부분이 없으면?? compile에서 에러 발생
  fn inner():
    print("hello mojo")
  outer(inner)

call_it()